In [1]:
import ROOT as root
import uproot
import matplotlib.pyplot as plt
import scipy as scp
from scipy.integrate import trapezoid
import numpy as np
import awkward as ak

Welcome to JupyROOT 6.30/04


In [8]:
f=root.TFile.Open("Dati/Run_Na_Couples_CDF20_Data_5_9_2024_Ascii.root", "READ")
myTree=f.Get("tree")
num_entries=myTree.GetEntries()
print(num_entries)
myTree.GetListOfBranches()
for branch in myTree.GetListOfBranches():
    branch_name = branch.GetName()
    branch_value = getattr(myTree, branch_name)
    print(f"    {branch_name}:  {branch_value}")
print()

100000
    ev:  0
    nch:  0
    ch:  <cppyy.LowLevelView object at 0x2a2090e30>
    base:  <cppyy.LowLevelView object at 0x2a2090d50>
    amp:  <cppyy.LowLevelView object at 0x2a2090f10>
    charge:  <cppyy.LowLevelView object at 0x2a2090f80>
    letime:  <cppyy.LowLevelView object at 0x2a2090ff0>
    tetime:  <cppyy.LowLevelView object at 0x2a2091060>
    ratecount:  <cppyy.LowLevelView object at 0x2a20910d0>
    pshape:  <cppyy.LowLevelView object at 0x2a2091140>



In [26]:
myTree.GetEntry(5)
var=getattr(myTree, "charge")
var[11]


-0.7429999709129333

In [3]:
keys = f.GetListOfKeys()
print("Contents of the ROOT file:")
for key in keys:
    obj = key.ReadObj()
    name = key.GetName()
    obj_class = obj.ClassName()
    print(f"Name: {name}, Class: {obj_class}")


Contents of the ROOT file:
Name: tree, Class: TTree
Name: tree, Class: TTree


In [19]:
myTree.GetEntry(0)
print(f"Entry {0}:")
# Loop over the branches and print their values
for branch in myTree.GetListOfBranches():
    branch_name = branch.GetName()
    branch_value = getattr(myTree, branch_name)
    print(f"    {branch_name}:  {branch_value}")
print()  # Print an empty line for better readability

Entry 0:
    ev:  1
    nch:  12
    ch:  <cppyy.LowLevelView object at 0x177f67b40>
    base:  <cppyy.LowLevelView object at 0x29648c260>
    amp:  <cppyy.LowLevelView object at 0x29648c7a0>
    charge:  <cppyy.LowLevelView object at 0x29648cb20>
    letime:  <cppyy.LowLevelView object at 0x29648cd50>
    tetime:  <cppyy.LowLevelView object at 0x29648cdc0>
    ratecount:  <cppyy.LowLevelView object at 0x29648ce30>
    pshape:  <cppyy.LowLevelView object at 0x29648cea0>



In [31]:
myTree.Show()

======> EVENT:0
 ev              = 1
 nch             = 12
 ch              = 4, 
                  5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 
                  15
 base            = 0.000252, 
                  0.000177, -3e-05, -5.7e-05, 4.2e-05, -0.000263, 
                  -0.008022, 0.000557, -0.003407, 0.000776, 0.000197, 
                  0.003689
 amp             = -0.002783, 
                  -0.002013, -0.002292, -0.002232, -0.187804, -0.109969, 
                  -0.005595, -0.00358, -0.002974, -0.002406, -0.002415, 
                  -0.00321
 charge          = -0.67, 
                  1.692, -0.146, 0.266, -374.247, -266.063, 
                  -1.472, -3.127, -0.873, -0.417, 0.115, 
                  -2.717
 letime          = 229.705, 
                  43.383, 49.445, 117.022, 65.175, 95.25, 
                  34.197, 310.66, 191.497, 278.911, 138.233, 
                  81.37
 tetime          = 230.449, 
                  44.033, 50.484, 118.745, 154.882, 224.738, 
       

In [5]:
d = root.RDataFrame("tree", "Dati/Run_Na_Couples_CDF20_Data_5_9_2024_Ascii.root")
d.GetColumnNames()
d.Display({"ch", "amp", "charge"}).Print()

+-----+-----+-------------+--------------+
| Row | ch  | charge      | amp          | 
+-----+-----+-------------+--------------+
| 0   | 4   | -0.670000f  | -0.00278300f | 
|     | 5   | 1.69200f    | -0.00201300f | 
|     | 6   | -0.146000f  | -0.00229200f | 
|     | 7   | 0.266000f   | -0.00223200f | 
|     | 8   | -374.247f   | -0.187804f   | 
|     | 9   | -266.063f   | -0.109969f   | 
|     | 10  | -1.47200f   | -0.00559500f | 
|     | 11  | -3.12700f   | -0.00358000f | 
|     | 12  | -0.873000f  | -0.00297400f | 
|     | 13  | -0.417000f  | -0.00240600f | 
|     | ... | ...         | ...          | 
+-----+-----+-------------+--------------+
| 1   | 4   | -1.48400f   | -0.00265900f | 
|     | 5   | 0.930000f   | -0.00353900f | 
|     | 6   | -1.49900f   | -0.00241100f | 
|     | 7   | 0.516000f   | -0.00220900f | 
|     | 8   | -525.583f   | -0.190956f   | 
|     | 9   | -448.495f   | -0.159751f   | 
|     | 10  | 0.731000f   | -0.00458600f | 
|     | 11  | -1.34600f   | -0.0023

In [48]:
cutb1 = lambda tree:tree.getattr(tree, "charge")< -100

cutb1 = d.Filter('amp>0.').Count()
#cutb1.Display({"ev"}).Print()
print(cutb1.GetValue())

AbortSignal: const ULong64_t& ROOT::RDF::RResultPtr<ULong64_t>::GetValue() =>
    AbortSignal: abort from C++; program state was reset

[/Users/martinachirico/root_build/lib/libcppyy_backend3_10.so] (anonymous namespace)::TExceptionHandlerImp::HandleException(int) (no debug info)
[/Users/martinachirico/root_build/lib/libCore.so] TUnixSystem::DispatchSignals(ESignals) (no debug info)
[/usr/lib/system/libsystem_platform.dylib] _sigtramp (no debug info)
[/usr/lib/system/libsystem_pthread.dylib] pthread_kill (no debug info)
[/usr/lib/system/libsystem_c.dylib] abort (no debug info)
[/usr/lib/libc++abi.dylib] __cxxabiv1::__aligned_malloc_with_fallback(unsigned long) (no debug info)
[/usr/lib/libc++abi.dylib] demangling_terminate_handler() (no debug info)
[/usr/lib/libobjc.A.dylib] _objc_terminate() (no debug info)
[/usr/lib/libc++abi.dylib] std::__terminate(void (*)()) (no debug info)
[/usr/lib/libc++abi.dylib] __cxa_get_exception_ptr (no debug info)
[/usr/lib/libc++abi.dylib] __cxxabiv1::exception_cleanup_func(_Unwind_Reason_Code, _Unwind_Exception*) (no debug info)
[/Users/martinachirico/root_build/lib/libROOTDataFrame.so]

In module 'ROOTDataFrame':
/Users/martinachirico/root_build/include/ROOT/RDF/InterfaceUtils.hxx:312:4: error: static_assert failed due to requirement 'std::is_convertible<ROOT::VecOps::RVec<int>, bool>::value' "filter expression returns a type that is not convertible to bool"
   static_assert(std::is_convertible<FilterRet_t, bool>::value,
   ^             ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/Users/martinachirico/root_build/include/ROOT/RDF/InterfaceUtils.hxx:450:4: note: in instantiation of function template specialization 'ROOT::Internal::RDF::CheckFilter<ROOT::VecOps::RVec<int> (ROOT::VecOps::RVec<float> &)>' requested here
   CheckFilter(f);
   ^
input_line_160:2:23: note: in instantiation of function template specialization 'ROOT::Internal::RDF::JitFilterHelper<ROOT::VecOps::RVec<int> (&)(ROOT::VecOps::RVec<float> &), ROOT::Detail::RDF::RNodeBase>' requested here
 ROOT::Internal::RDF::JitFilterHelper(R_rdf::func7, new const char*[1]{"amp"}, 1, "", reinterpret_cast<std::we